In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()

In [5]:
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [6]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()

In [8]:
train_mean, train_std

(tensor(-6.2598e-06), tensor(1.))

In [9]:
#export
def test_near_zero(a, tol=1e-3):
    assert a.abs() < tol, f'Near zero: {a}'

In [10]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [11]:
n,m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

## Foundations version

In [12]:
nh = 50

In [13]:
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [14]:
test_near_zero(w1.mean())

In [15]:
test_near_zero(w1.std() - 1 / math.sqrt(m))

In [16]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [17]:
def lin(x, w, b):
    return x @ w + b

In [18]:
t = lin(x_valid, w1, b1)

In [19]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0252), tensor(0.9935))

In [20]:
def relu(x):
    return x.clamp_min(0.)

In [21]:
t = relu(lin(x_valid, w1, b1))

In [22]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.4058), tensor(0.5841))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [23]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)

In [24]:
w1.mean(), w1.std()

(tensor(0.0002), tensor(0.0505))

In [25]:
t = relu(lin(x_valid, w1, b1))

In [26]:
t.mean(),t.std()

(tensor(0.5518), tensor(0.8328))

In [27]:
w1.shape

torch.Size([784, 50])

In [28]:
#export
from torch.nn import init

In [29]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [30]:
init.kaiming_normal_??

In [31]:
w1.mean(), w1.std()

(tensor(0.0003), tensor(0.0506))

In [32]:
t.mean(), t.std()

(tensor(0.5196), tensor(0.8041))

In [33]:
w1.shape

torch.Size([784, 50])

In [34]:
import torch.nn

In [35]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

In [36]:
torch.nn.Linear.forward??

In [37]:
torch.nn.functional.linear??

In [38]:
torch.nn.Conv2d??

In [39]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [40]:
# what if?
def relu(x):
     return x.clamp_min(0.) - 0.5

In [41]:
w1 = torch.randn(m, nh) * math.sqrt(2. / m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(-0.0067), tensor(0.7880))

In [42]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 =  lin(l2, w2, b2)
    return l3

In [43]:
%timeit -n 10 _=model(x_valid)

2.66 ms ± 405 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

### Loss function: MSE

In [45]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [46]:
# export
def mse(output, target):
    return ((output.squeeze(-1) - target).pow(2)).mean()

In [47]:
y_train, y_valid = y_train.float(), y_valid.float()

In [48]:
preds = model(x_train)

In [49]:
preds.shape

torch.Size([50000, 1])

In [50]:
y_train.shape

torch.Size([50000])

In [51]:
mse(preds, y_train)

tensor(26.0749)

### Gradients and backward pass

In [57]:
w1.shape

torch.Size([784, 50])

In [58]:
l1 = x_train @ w1 + b1
l1.shape

torch.Size([50000, 50])

In [60]:
l2 = relu(l1)
l2.shape

torch.Size([50000, 50])

In [62]:
w2.shape

torch.Size([50, 1])

In [63]:
out = l2 @ w2 + b2
out.shape

torch.Size([50000, 1])

In [65]:
mse_grad_ = 2. * (out.squeeze(-1) - y_train).unsqueeze(-1) / out.shape[0]

In [67]:
mse_grad_.shape

torch.Size([50000, 1])

In [68]:
mse_grad_[:10]

tensor([[-1.7756e-04],
        [ 1.2907e-05],
        [-1.8068e-04],
        [-4.9977e-05],
        [-3.4041e-04],
        [-5.8920e-05],
        [-1.2690e-05],
        [-1.1787e-04],
        [-3.8138e-05],
        [-1.7217e-04]])

In [52]:
def mse_grad(inp, target):
    # gradient of the loss with respect to the output of previous layer
    inp.g = 2. * (inp.squeeze(-1) - target).unsqueeze(-1) / inp.shape[0]

In [53]:
def relu_grad(inp, out):
    # gradient of relu with respect to input activations
    inp.g = (inp > 0).float() * out.g

In [54]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [55]:
def forward_and_backward(inp, target):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    mse_grad(out, target)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [56]:
forward_and_backward(x_train, y_train)

In [57]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [58]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [59]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [60]:
loss = forward(xt2, y_train)

In [61]:
loss.backward()

In [62]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor Model

### Layers as classes

In [63]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [64]:
class Lin():
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [65]:
class Mse():
    def __call__(self, inp, target):
        self.inp = inp
        self.target = target
        self.out = (inp.squeeze(-1) - target).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze(-1) - self.target).unsqueeze(-1) / self.inp.shape[0]

In [66]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [67]:
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model(w1, b1, w2, b2)

In [68]:
%time loss = model(x_train, y_train)

CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 20.8 ms


In [69]:
%time model.backward()

CPU times: user 3.7 s, sys: 5.29 s, total: 8.99 s
Wall time: 2.32 s


In [70]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [71]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):
        raise Exception("Not implemented")
        
    def backward(self):
        self.bwd(self.out, *self.args)

In [72]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.) - 0.5
    
    def bwd(self, out, inp):
        inp.g = (inp > 0).float() * out.g

In [85]:
class Lin(Module):
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def forward(self, inp):
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
#         self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [86]:
class Mse(Module):
    def forward(self, inp, target):
        return (inp.squeeze(-1) - target).pow(2).mean()
    
    def bwd(self, out, inp, target):
        inp.g = 2. * (inp.squeeze(-1) - target).unsqueeze(-1) / inp.shape[0]

In [87]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [88]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [89]:
%time loss = model(x_train, y_train)

CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 21.5 ms


In [90]:
%time model.backward()

CPU times: user 220 ms, sys: 160 ms, total: 380 ms
Wall time: 94.9 ms


In [91]:
test_near(w2g, w2.g)

In [92]:
test_near(b2g, b2.g)

In [93]:
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [94]:
class Lin(Module):
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def forward(self, inp):
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [95]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [96]:
%time loss = model(x_train, y_train)

CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 21.6 ms


In [97]:
%time model.backward()

CPU times: user 252 ms, sys: 128 ms, total: 380 ms
Wall time: 95.1 ms


In [98]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [99]:
#export
from torch import nn

In [122]:
class Model(nn.Module):
    def __init__(self, n_in, n_h, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, n_h), nn.ReLU(), nn.Linear(n_h, n_out)]
        self.loss = mse
        
    def __call__(self, x, target):
        for l in self.layers:
            x = l(x)
        return self.loss(x, target)

In [123]:
model = Model(m, nh, 1)

In [124]:
%time loss = model(x_train, y_train)

CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 20.6 ms


In [125]:
%time loss.backward()

CPU times: user 84 ms, sys: 8 ms, total: 92 ms
Wall time: 23.9 ms


In [143]:
for param in model.layers[0].parameters():
    print(param)

Parameter containing:
tensor([[-0.0192, -0.0350, -0.0222,  ...,  0.0213, -0.0098, -0.0040],
        [-0.0327, -0.0172, -0.0324,  ...,  0.0240,  0.0357,  0.0282],
        [-0.0305,  0.0295,  0.0139,  ...,  0.0253,  0.0053, -0.0007],
        ...,
        [ 0.0327, -0.0173,  0.0245,  ..., -0.0111, -0.0075, -0.0278],
        [ 0.0114, -0.0320,  0.0134,  ...,  0.0274,  0.0198,  0.0294],
        [-0.0127, -0.0018, -0.0348,  ..., -0.0206,  0.0146, -0.0004]],
       requires_grad=True)
Parameter containing:
tensor([-0.0128, -0.0087,  0.0030,  0.0352, -0.0309, -0.0197,  0.0067, -0.0035,
        -0.0104, -0.0059, -0.0342, -0.0108,  0.0243,  0.0027,  0.0015,  0.0263,
        -0.0004,  0.0144,  0.0038, -0.0013, -0.0123, -0.0157,  0.0258,  0.0058,
         0.0219,  0.0006,  0.0295,  0.0307, -0.0208, -0.0265,  0.0090, -0.0290,
        -0.0302,  0.0284, -0.0232,  0.0111,  0.0192, -0.0324,  0.0008,  0.0277,
         0.0321,  0.0146,  0.0347, -0.0113,  0.0066, -0.0142,  0.0153, -0.0070,
         0.0070

In [145]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
